<a href="https://colab.research.google.com/github/liam-0/Most-difficult-problem-for-Logic-and-Language/blob/main/NLI_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Do run this page in GPU mode**

This colab page is for L&L final task.

Group members: \\
Yanming Li (2033070) \\
Joep Nouwens (6537618) \\
Mitchell (5720664) \\
Hugo van Meeteren (6568505)

# Data preparation

In order to enable humans can manually check the wrong cases of models' predictions and find their patterns. This section is for getting the result of the three models with SNLI and Multi-NLI datasets. And save it as .csv files. 

## Model set up

In [ ]:
!pip install datasets
!pip install transformers
!pip install SentencePiece

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from datasets import load_dataset
import pandas as pd
import sys
import time
from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.2 MB/s eta 0:00:00


In [ ]:
# First model:
# ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli
# https://huggingface.co/ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli

# Second model:
# cross-encoder/nli-deberta-v3-large
# https://huggingface.co/cross-encoder/nli-deberta-v3-large?candidateLabels=contradiction%2C+entailment%2C+neutral&multiClass=false&text=A+man+is+eating+pizza.+A+man+eats+something

# Third model:
# cross-encoder/nli-MiniLM2-L6-H768
# https://huggingface.co/cross-encoder/nli-MiniLM2-L6-H768?candidateLabels=contradiction%2C+entailment%2C+neutral&multiClass=false&text=A+man+is+eating+pizza.+A+man+eats+something

def model_setup(model_name):
  model = AutoModelForSequenceClassification.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  #
  return model, tokenizer


ynie_model, ynie_tokenizer = model_setup(
    'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli')
deberta_v3_large_model, deberta_v3_large_tokenizer = model_setup(
    'cross-encoder/nli-deberta-v3-large')
nli_MiniLM2_L6_H768_model, nli_MiniLM2_L6_H768_tokenizer = model_setup(
    'cross-encoder/nli-MiniLM2-L6-H768')


Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Loading dataset

In [ ]:
# These two datasets' details can be seen in the slides of the 17/1/2023 lecture
snli_dataset = load_dataset("snli")
# snli_dataset have 10000 rows of data
# https://huggingface.co/datasets/snli
# use snli_dataset['test'] to get to the data
# 0: entailment 1: neutral 2: contradiction


multi_nli_dataset = load_dataset("multi_nli")
# multi_nli_dataset have 9815 rows of data for validation_matched and 9832 rows of data for validation_mismatched
# https://huggingface.co/datasets/multi_nli/viewer/default/validation_matched
# use multi_nli_dataset['validation_matched'] and multi_nli_dataset['validation_mismatched'] to get to the data
# 0: entailment 1: neutral 2: contradiction


Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


...

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def dataset2df(dataset):
  if 'sentence1' in dataset.features:
    premise_name = "sentence1"
    hypothesis_name = "sentence2"
    label_name = "gold_label"
  else:
    premise_name = "premise"
    hypothesis_name = "hypothesis"
    label_name = "label"

  premise_data = list(dataset[premise_name])
  hypothesis_data = list(dataset[hypothesis_name])
  label_data = list(dataset[label_name])

  dict_data = {'premise': premise_data,
               'hypothesis': hypothesis_data, 'label': label_data}

  temp_output = pd.DataFrame(dict_data)

  temp_output.replace(0, "entailment", inplace=True)
  temp_output.replace(1, "neutral", inplace=True)
  temp_output.replace(2, "contradiction", inplace=True)

  return temp_output


df_snli_test_dataset = dataset2df(snli_dataset['test'])
df_multi_nli_validation_matched_dataset = dataset2df(
    multi_nli_dataset['validation_matched'])
df_multi_nli_validation_mismatched_dataset = dataset2df(
    multi_nli_dataset['validation_mismatched'])


## Model prediction

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


def model_predict(model, tokenizer, dataset, is_ynie=0):
  total_num = len(dataset)
  prediction = []
  is_correct = []
  max_score = []
  contradiction_score = []
  entailment_score = []
  neutral_score = []

  # for row in dataset.values:
  for index, row in enumerate(dataset.values):
    features = tokenizer([row[0]], [row[1]], padding=True,
                         truncation=True, return_tensors="pt").to(device)
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        scores = model(**features).logits
        scores = torch.softmax(scores, dim=1).tolist()[0]
        # The label order does affect the predicted result
        if is_ynie == 0:

          label_mapping = ['contradiction', 'entailment', 'neutral']
          contradiction_score.append(scores[0])
          entailment_score.append(scores[1])
          neutral_score.append(scores[2])

        else:
          label_mapping = ['entailment', 'neutral', 'contradiction']
          entailment_score.append(scores[0])
          neutral_score.append(scores[1])
          contradiction_score.append(scores[2])
        # labels = [label_mapping[score_max] for score_max in scores.argmax(dim=1)]
        labels = label_mapping[scores.index(max(scores))]
        max_score.append(max(scores))

        prediction.append(labels)

        if row[2] == labels:
          is_correct.append(1)
        else:
          is_correct.append(0)

    print("\r", end="")
    print("Progress: {}/{}".format(index+1, total_num), end="")
    sys.stdout.flush()
    time.sleep(0.001)

  dict_data = {'prediction': prediction, 'is_correct': is_correct, 'max_score': max_score,
               'contradiction_score': contradiction_score, 'entailment_score': entailment_score, 'neutral_score': neutral_score}

  df_dict_data = pd.DataFrame(dict_data)
  df_dict_data.index = dataset.index

  temp_output = pd.concat([dataset, df_dict_data], axis=1)

  return temp_output


In [ ]:
#################################################################################
# Save model results to .csv files and download it.
#################################################################################
print("Model: ynie, Dataset: snli test")
res_ynie_snli_test = model_predict(
    ynie_model, ynie_tokenizer, df_snli_test_dataset, is_ynie=1)
res_ynie_snli_test.to_csv('res_ynie_snli_test.csv', encoding='utf-8-sig')
files.download('res_ynie_snli_test.csv')
print(" ")

print("Model: ynie, Dataset: multi nli validation matched")
res_ynie_multi_nli_validation_matched = model_predict(
    ynie_model, ynie_tokenizer, df_multi_nli_validation_matched_dataset, is_ynie=1)
res_ynie_multi_nli_validation_matched.to_csv(
    'res_ynie_multi_nli_validation_matched.csv', encoding='utf-8-sig')
files.download('res_ynie_multi_nli_validation_matched.csv')
print(" ")


print("Model: ynie, Dataset: multi nli validation mismatched")
res_ynie_multi_nli_validation_mismatched = model_predict(
    ynie_model, ynie_tokenizer, df_multi_nli_validation_mismatched_dataset, is_ynie=1)
res_ynie_multi_nli_validation_mismatched.to_csv(
    'res_ynie_multi_nli_validation_mismatched.csv', encoding='utf-8-sig')
files.download('res_ynie_multi_nli_validation_mismatched.csv')
print(" ")

################################################################################
print("Model: deberta v3 large, Dataset: snli test")
res_deberta_v3_large_snli_test = model_predict(
    deberta_v3_large_model, deberta_v3_large_tokenizer, df_snli_test_dataset, is_ynie=0)
res_deberta_v3_large_snli_test.to_csv(
    'res_deberta_v3_large_snli_test.csv', encoding='utf-8-sig')
files.download('res_deberta_v3_large_snli_test.csv')
print(" ")

print("Model: deberta v3 large, Dataset: multi nli validation matched")
res_deberta_v3_large_multi_nli_validation_matched = model_predict(
    deberta_v3_large_model, deberta_v3_large_tokenizer, df_multi_nli_validation_matched_dataset, is_ynie=0)
res_deberta_v3_large_multi_nli_validation_matched.to_csv(
    'res_deberta_v3_large_multi_nli_validation_matched.csv', encoding='utf-8-sig')
files.download('res_deberta_v3_large_multi_nli_validation_matched.csv')
print(" ")


print("Model: deberta v3 large, Dataset: multi nli validation mismatched")
res_deberta_v3_large_multi_nli_validation_mismatched = model_predict(
    deberta_v3_large_model, deberta_v3_large_tokenizer, df_multi_nli_validation_mismatched_dataset, is_ynie=0)
res_deberta_v3_large_multi_nli_validation_mismatched.to_csv(
    'res_deberta_v3_large_multi_nli_validation_mismatched.csv', encoding='utf-8-sig')
files.download('res_deberta_v3_large_multi_nli_validation_mismatched.csv')
print(" ")

################################################################################
print("Model: nli MiniLM2 L6 H768, Dataset: snli test")
res_nli_MiniLM2_L6_H768_snli_test = model_predict(
    nli_MiniLM2_L6_H768_model, nli_MiniLM2_L6_H768_tokenizer, df_snli_test_dataset, is_ynie=0)
res_nli_MiniLM2_L6_H768_snli_test.to_csv(
    'res_nli_MiniLM2_L6_H768_snli_test.csv', encoding='utf-8-sig')
files.download('res_nli_MiniLM2_L6_H768_snli_test.csv')
print(" ")

print("Model: nli MiniLM2 L6 H768, Dataset: multi nli validation matched")
res_nli_MiniLM2_L6_H768_multi_nli_validation_matched = model_predict(
    deberta_v3_large_model, deberta_v3_large_tokenizer, df_multi_nli_validation_matched_dataset, is_ynie=0)
res_nli_MiniLM2_L6_H768_multi_nli_validation_matched.to_csv(
    'res_nli_MiniLM2_L6_H768_multi_nli_validation_matched.csv', encoding='utf-8-sig')
files.download('res_nli_MiniLM2_L6_H768_multi_nli_validation_matched.csv')
print(" ")

print("Model: nli MiniLM2 L6 H768, Dataset: multi nli validation mismatched")
res_nli_MiniLM2_L6_H768_multi_nli_validation_mismatched = model_predict(
    deberta_v3_large_model, deberta_v3_large_tokenizer, df_multi_nli_validation_mismatched_dataset, is_ynie=0)
res_nli_MiniLM2_L6_H768_multi_nli_validation_mismatched.to_csv(
    'res_nli_MiniLM2_L6_H768_multi_nli_validation_mismatched.csv', encoding='utf-8-sig')
files.download('res_nli_MiniLM2_L6_H768_multi_nli_validation_mismatched.csv')
print(" ")


Model: ynie, Dataset: snli test
Progress: 10000/10000

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 
Model: ynie, Dataset: multi nli validation matched
Progress: 9815/9815

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 
Model: nli MiniLM2 L6 H768, Dataset: multi nli validation mismatched
Progress: 9832/9832

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

result examples

In [ ]:
res_ynie_snli_test[res_ynie_snli_test['is_correct']== 0]

,premise,hypothesis,label,prediction,is_correct,max_score,contradiction_score,entailment_score,neutral_score
4,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,entailment,neutral,0,0.718575,0.003076,0.278349,0.718575
12,A land rover is being driven across a river.,A Land Rover is splashing water as it crosses ...,entailment,neutral,0,0.783293,0.004057,0.212649,0.783293
43,Male in a blue jacket decides to lay in the gr...,The guy wearing a blue jacket is laying on the...,entailment,neutral,0,0.970083,0.001174,0.028744,0.970083
46,During calf roping a cowboy calls off his horse.,Cowboy falling off horse.,-1,contradiction,0,0.574513,0.574513,0.058600,0.366887
47,During calf roping a cowboy calls off his horse.,A man ropes a calf successfully.,contradiction,neutral,0,0.852201,0.078653,0.069146,0.852201
...,...,...,...,...,...,...,...,...,...
9955,Four dark-skinned boys are playing soccer.,The boy is near a ball.,-1,entailment,0,0.939672,0.004497,0.939672,0.055831
9959,A white duck is spreading its wings while sitt...,The duck kept it's wings intact when it dived ...,contradiction,neutral,0,0.582830,0.398680,0.018490,0.582830
9962,A little boy is getting is birthday cake and i...,A child at a party.,neutral,entailment,0,0.742990,0.001346,0.742990,0.255664
9965,A group of girls jumping over another girl who...,A gorup of girl is laying down.,contradiction,entailment,0,0.860792,0.024635,0.860792,0.114573


#Data analysis, and automatic filter design

In [1]:
!pip install pyenchant
!pip install vadersentiment
# !pip install Caribe
!pip install language-tool-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 2.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 KB 3.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import enchant
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import sklearn.metrics
import pandas as pd
# import Caribe as cb
import language_tool_python

In [73]:
def get_check_dataset(path):
  ###############################################################################
  # get_check_dataset from created .csv file and creat a desired dataset
  # where check_sentence should be one of "all", "difficult", "easy"
  # "difficult" means only check difficult sentences, "easy" means only check easy sentences, "all" means check all sentences
  ###############################################################################

  df_dataset = pd.read_csv(path, delimiter=';', skiprows=0, low_memory=False)
  # seprate the dataset into two parts: difficult and easy
  # where difficult is the cases where 3 models all predicted wrong, easy is the case where all models predicted right
  check_diff = df_dataset.loc[df_dataset["test"] == 0, ["premise", "hypothesis"]]
  check_easy = df_dataset.loc[df_dataset["test"] == 3, ["premise", "hypothesis"]]

  check = pd.concat([check_diff,check_easy])

  check_label = ["difficult"] * len(check_diff) + ["easy"] * len(check_easy)
  check = check.reset_index()

  return check[["premise","hypothesis"]].copy(deep=True), check_label

In [52]:
def rule_based_system(check_dataset, check_label):
  ###############################################################################
  # This function is applying rule based model, and give the accuracy of rule based system
  # input: the testing datset, and its labels
  # return: the result of rule based system
  ###############################################################################
  result = []
  enchant_dict = enchant.Dict("en_US") # for rule 1
  my_tool = language_tool_python.LanguageTool('en-US') # for rule 3

  # test1 = 0 # for determining min sentiment value (rule 2)
  # test2 = 0 # for determining min sentiment value (rule 2)

  for index, row in check_dataset.iterrows():
    hypo = row["hypothesis"]
    hypo.replace(', ', ' ') # check function can't work with comma
    prem = row["premise"]
    prem.replace(', ', ' ') # check function can't work with comma
    temp = 0
  
  # TO DETERMINE MIN/MAX SENTIMENT VALUE (rule 2)
    # for word in hypo.split(): # check every word ...
    #   if SentimentIntensityAnalyzer().polarity_scores(word)["compound"] < test1:
    #     test1 = SentimentIntensityAnalyzer().polarity_scores(word)["compound"]
    #   if SentimentIntensityAnalyzer().polarity_scores(word)["compound"] > test2:
    #     test2 = SentimentIntensityAnalyzer().polarity_scores(word)["compound"]

    # RULE 1
    # for word in hypo.split(): # check every word ...
    #   if not enchant_dict.check(word):
    #     temp = 1
    #     result.append("difficult")
    #     break

    # if temp == 0:
    #   result.append("easy")

    # RULE 2
    # for word in hypo.split(): # check every word ...
    #   if SentimentIntensityAnalyzer().polarity_scores(word)["compound"] < 0:
    #     temp = 1
    #     result.append("difficult")
    #     break

    # if temp == 0:
    #   result.append("easy")

    # RULE 3
    # my_matches_hypo = my_tool.check(hypo)
    # if my_matches_hypo:
    #     temp = 1
    #     result.append("difficult")
    #     break

    # if temp == 0:
    #   result.append("easy")

    # RULE 4
    # my_matches_prem = my_tool.check(prem)
    # if my_matches_prem:
    #     temp = 1
    #     result.append("difficult")
    #     break

    # if temp == 0:
    #   result.append("easy")

    # RULE 2 + 4 (targeting)
    # my_matches_prem = my_tool.check(prem)
    # if my_matches_prem:
    #   for word in hypo.split(): # check every word ...
    #     if SentimentIntensityAnalyzer().polarity_scores(word)["compound"] == -0.5574:
    #       temp = 1
    #       result.append("difficult")
    #       break

    # if temp == 0:
    #   result.append("easy")

    # ALL RULES COMBINED + SPECIFIC SENTIMENT VALUES
    for word in hypo.split(): # check every word ...
      if not enchant_dict.check(word): # SentimentIntensityAnalyzer().polarity_scores(c)["compound"] <= -0.5
        if temp == 0:
          my_matches_hypo = my_tool.check(hypo)
          my_matches_prem = my_tool.check(prem)
          if my_matches_hypo and my_matches_prem:
            temp = 1
          else:
            break

    if temp == 1:
      for new_word in hypo.split():
        if SentimentIntensityAnalyzer().polarity_scores(new_word)["compound"] >= 0 and SentimentIntensityAnalyzer().polarity_scores(new_word)["compound"] <= 0.5719: # (RULE 1) if there is a non-gramatical word ... not enchant_dict.check(work) 
          temp = 1
        else:
          temp = 0
          break

    if temp == 1:
        result.append("difficult")

    if temp == 0:
        result.append("easy")
    
  print(sklearn.metrics.accuracy_score(check_label, result))
  return result

In [97]:
path = "/content/res_ynie_snli_test_(combined_snli).csv"
# path = "/content/res_ynie_multi_nli_validation_matched_(combined_mnli_matched).csv"
# path = "/content/res_ynie_multi_nli_validation_mismatched_(combined_mnli_mismatched).csv"

check, check_label = get_check_dataset(path)
result = rule_based_system(check, check_label)

0.949192462987887
